In [1]:
#from mpl_toolkits.basemap import Basemap
from matplotlib import pyplot as plt
import pandas as pd
import sklearn

In [2]:
data = pd.read_csv("property assessment 2022.csv")

differences between properties: 
neighborhood, price/quality correlations, correlations between features of property and price
segmentation, cluster analysis

In [3]:
data

,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,UNIT_NUM,CITY,ZIPCODE,BLDG_SEQ,NUM_BLDGS,...,KITCHEN_STYLE2,KITCHEN_STYLE3,HEAT_TYPE,HEAT_FUEL,AC_TYPE,FIRE_PLACE,ORIENTATION,NUM_PARKING,PROP_VIEW,CORNER_UNIT
0,100001000,,1.000010e+08,104 A 104,PUTNAM ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,3,A - Average,NaN
1,100002000,,1.000020e+08,197,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,M - Modern,M - Modern,F - Forced Hot Air,NaN,C - Central AC,0,NaN,0,A - Average,NaN
2,100003000,,1.000030e+08,199,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,S - Space Heat,NaN,N - None,0,NaN,0,A - Average,NaN
3,100004000,,1.000040e+08,201,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,0,A - Average,NaN
4,100005000,,1.000050e+08,203,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,NaN,W - Ht Water/Steam,NaN,N - None,0,NaN,0,A - Average,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178593,2205666000,,2.205666e+09,,KNOWLES ST,NaN,BRIGHTON,2135,1.0,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN
178594,2205667000,,2.205667e+09,,LAKE ST,NaN,BRIGHTON,2135,1.0,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN
178595,2205668000,,2.205668e+09,4,LAKE ST,NaN,BRIGHTON,2135,1.0,1,...,M - Modern,M - Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,2,A - Average,NaN
178596,2205669000,,2.205669e+09,2193 2201,COMMONWEALTH AV,NaN,BRIGHTON,2135,1.0,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN


In [4]:
data.columns

Index(['PID', 'CM_ID', 'GIS_ID', 'ST_NUM', 'ST_NAME', 'UNIT_NUM', 'CITY',
       'ZIPCODE', 'BLDG_SEQ', 'NUM_BLDGS', 'LUC', 'LU', 'LU_DESC', 'BLDG_TYPE',
       'OWN_OCC', 'OWNER', 'MAIL_ADDRESSEE', 'MAIL_ADDRESS', 'MAIL_CITY',
       'MAIL_STATE', 'MAIL_ZIPCODE', 'RES_FLOOR', 'CD_FLOOR', 'RES_UNITS',
       'COM_UNITS', 'RC_UNITS', 'LAND_SF', 'GROSS_AREA', 'LIVING_AREA',
       'LAND_VALUE', 'BLDG_VALUE', 'TOTAL_VALUE', 'GROSS_TAX', 'YR_BUILT',
       'YR_REMODEL', 'STRUCTURE_CLASS', 'ROOF_STRUCTURE', 'ROOF_COVER',
       'INT_WALL', 'EXT_FINISHED', 'INT_COND', 'EXT_COND', 'OVERALL_COND',
       'BED_RMS', 'FULL_BTH', 'HLF_BTH', 'KITCHEN', 'TT_RMS', 'BDRM_COND',
       'BTHRM_STYLE1', 'BTHRM_STYLE2', 'BTHRM_STYLE3', 'KITCHEN_TYPE',
       'KITCHEN_STYLE1', 'KITCHEN_STYLE2', 'KITCHEN_STYLE3', 'HEAT_TYPE',
       'HEAT_FUEL', 'AC_TYPE', 'FIRE_PLACE', 'ORIENTATION', 'NUM_PARKING',
       'PROP_VIEW', 'CORNER_UNIT'],
      dtype='object')

Just a meaningless cell to play with the groupby function.

In [5]:
#print(data.groupby("ZIPCODE").describe().columns)
#data.groupby("ZIPCODE").agg(['sum'])

The following code drops the columns and data we absolutely don't need - such as IDs and duplicate addresses and mailing info.

In [6]:
cleaned = data.drop(['CM_ID', 'GIS_ID','UNIT_NUM','BLDG_SEQ','LUC','LU_DESC','CITY',"OWNER","MAIL_ADDRESSEE",'MAIL_ADDRESS', 'MAIL_CITY','MAIL_STATE', 'MAIL_ZIPCODE',"ORIENTATION"],axis=1)

Next, we need to create mappings for each categorical variable and fill in missing data in a reasonable way. For example, missing data for kitchens is equivalent to not having one at all.

mapping = {"a" : 1...}
df["col"] = df["col"].map(mapping)

This is an example of how to find the possible values in each column in order to figure out how to best fill in the data.

In [7]:
cleaned['STRUCTURE_CLASS'].unique()
#data.isnull().sum().to_string()
#for x in data.isnull().sum():
#  print(x)

array(['R - ', nan, 'D - Wood/Frame', 'C - Brick/Concr',
       'B - Reinf Concr', 'A - Struct Steel', 'E - Metal'], dtype=object)

In [8]:
# drop properties without BLDG_VALUE
# for GROSS_AREA and LIVING_AREA -
  # LIVING_AREA is often null for business type buildings, or condo management buildings
  # GROSS_AREA seems to just be a case of missing data?
  # this requires more investigation, but for now, dropping all properties with no gross area also appears to drop all those with
  # no living area either (meaning that all properties with no living area data also are missing gross area data)
# replace nan in BLDG_TYPE with NoBld
# for kitchens - get the median type of kitchen and keep the count of kitchens rather than having 3 columns for three types of kitchens
# replace NAN in HEAT_FUEL with N - None
# same for HEAT_TYPE, AC_TYPE
# for FIRE_PLACE - simply assume nan = no fireplace
# for bathrooms, average out the style of bathroom
# merge res_floor and cd_floor
# for roof strucutre - change nan to "O - Other"
# for Structure class 
# for NUM_PARKING - assume there nan is no parking
# merge RES_UNITS, COM_UNITS, RC_UNITS
# for anything else, data can probably be filled in with median for continuous or mode for categorical

Cleaning up the data

Let's see where we have missing values.

In [9]:
cleaned.isnull().sum()

PID                     0
ST_NUM                  0
ST_NAME                 0
ZIPCODE                 0
NUM_BLDGS               0
LU                      0
BLDG_TYPE            2504
OWN_OCC                 0
RES_FLOOR           23324
CD_FLOOR           109709
RES_UNITS               0
COM_UNITS               0
RC_UNITS                0
LAND_SF             80768
GROSS_AREA          23198
LIVING_AREA         23198
LAND_VALUE            176
BLDG_VALUE            176
TOTAL_VALUE           176
GROSS_TAX             176
YR_BUILT            23198
YR_REMODEL          86506
STRUCTURE_CLASS     99138
ROOF_STRUCTURE      36738
ROOF_COVER          36739
INT_WALL            47646
EXT_FINISHED        23289
INT_COND            47639
EXT_COND            36708
OVERALL_COND         9325
BED_RMS                 0
FULL_BTH                0
HLF_BTH                 0
KITCHEN                 0
TT_RMS                  0
BDRM_COND          110004
BTHRM_STYLE1        47220
BTHRM_STYLE2        94416
BTHRM_STYLE3

First, let's drop properties with no BLDG_VALUE.

In [10]:
cleaned = cleaned.drop(data.index[data["BLDG_VALUE"].isnull()])
print(len(cleaned))

178422


Next, we need to handle the fact that many properties are missing a combination of land size, gross property size, as well as living space. Possible reasons for this include the fact that condominum units might not have a land size or gross property size, but will have a living space size. Commerical properties might have no living space but have a gross property size instead.

The best way to figure this out is to look at properties with one of those values missing and then see what the most common land use for that type is.

In [11]:
cleaned[cleaned["GROSS_AREA"].isnull()]["LU"].value_counts()

CM    10010
RL     6150
E      2508
CP     1880
CC     1206
C       476
CL      405
A       272
RC      133
I        55
EA       46
R4       40
CD       14
AH        2
R1        1
Name: LU, dtype: int64

The vast majority of parcels with no gross floor space are condominum mains representing all units and with no actual value.

In [12]:
cleaned[cleaned["LIVING_AREA"].isnull()]["LU"].value_counts()

CM    10010
RL     6150
E      2508
CP     1880
CC     1206
C       476
CL      405
A       272
RC      133
I        55
EA       46
R4       40
CD       14
AH        2
R1        1
Name: LU, dtype: int64

Given the that counts for each type of land use are identical for both gross area and living area, it's probable that they are the exact same parcels. Given that they seem to be missing this data and there's no good way to fill it in, they might just have to be dropped entirely. 

An alternate option is to keep them, but mark 0 for both. This makes sense for LIVING_AREA, but it does not make sense for GROSS_AREA.

In [13]:
cleaned[cleaned["LAND_SF"].isnull()]["LU"].value_counts()

CD    68604
CP     7646
CC     1507
C       960
E       737
A       468
RC      292
CL       93
I        72
CM       58
RL       50
R4       48
EA       26
R1       26
R3        7
R2        4
Name: LU, dtype: int64

For land size, it's mainly condominum units that have no land size. This makes sense, as those properties are just the housing and nothing more. In this case, missing land data for condominium units can actually be filled in with 0 because they truly have no land. This reasoning probably applies to all of the other types except parking, and those can just be dropped.

This allows us to keep most of the rows with missing LAND_SF data.

In [14]:
cleaned["LAND_SF"] = cleaned["LAND_SF"].fillna(0)
cleaned.drop(data.index[data["GROSS_AREA"].isnull()],inplace=True)

In [15]:
cleaned.isnull().sum()

PID                     0
ST_NUM                  0
ST_NAME                 0
ZIPCODE                 0
NUM_BLDGS               0
LU                      0
BLDG_TYPE             967
OWN_OCC                 0
RES_FLOOR           11016
CD_FLOOR            86339
RES_UNITS               0
COM_UNITS               0
RC_UNITS                0
LAND_SF                 0
GROSS_AREA              0
LIVING_AREA             0
LAND_VALUE              0
BLDG_VALUE              0
TOTAL_VALUE             0
GROSS_TAX               0
YR_BUILT            10905
YR_REMODEL          71701
STRUCTURE_CLASS     77802
ROOF_STRUCTURE      24454
ROOF_COVER          24455
INT_WALL            24811
EXT_FINISHED        11047
INT_COND            24815
EXT_COND            24424
OVERALL_COND         7089
BED_RMS                 0
FULL_BTH                0
HLF_BTH                 0
KITCHEN                 0
TT_RMS                  0
BDRM_COND           86634
BTHRM_STYLE1        24725
BTHRM_STYLE2        71840
BTHRM_STYLE3

There are now no missing values in LAND_SF, GROSS_AREA, or LIVING_AREA.

We now have to deal with the missing BLDG_TYPE. Let's see what's common, since this will likely have to be replaced with a dummy value or the mode.

In [16]:
cleaned['BLDG_TYPE'].unique()

array(['RE - Row End', 'RM - Row Middle', 'DK - Decker',
       'CV - Conventional', 'TF - Two Fam Stack', 'SD - Semi-Det',
       'CL - Colonial', 'CP - Cape', '111 - APT 4-6 UNITS',
       '327 - RESTAURANT/Lounge', '026 - RC: TWO RES UNITS',
       'DX - Duplex', '333 - SELF-SERV STATION', '320 - RET/WHSL/SERVICE',
       '325 - RETAIL STORE Detached', '025 - RC: ONE RES UNIT',
       '343 - OFFICE 1-2 STORY', '027 - RC: THREE RES UNITS',
       'VT - Victorian', '112 - APT 7-30 UNITS', '332 - REPAIR GARAGE',
       'TH - Town House', 'FS - Free Standing', nan,
       '316 - WAREHOUSE /Distrib', '326 - RESTAURANT/Cafeteria',
       'NoBld - ', '319 - STRIP RETAIL/ OFFICE', '99 - Vacant',
       '342 - MEDICAL OFFICE', '397 - BUS /RR TERMINAL',
       '312 - Self-Storage whse', '317 - STORAGE WHSE (old)',
       '446 - STEAM/WATER/SEWER', '335 - CAR WASH',
       '328 - FAST FOOD Restaurant', '344 - OFFICE 3-9 STORY',
       '300 - HOTEL', 'LR - Low Rise', '117 - DAY CARE CENTER',
  

There's a lot of choices here. Since LU (Land Use) is valid for every parcel, let's see what the most common land use is for these buildings.

In [17]:
cleaned[cleaned["BLDG_TYPE"].isnull()]["LU"].value_counts()

CL    880
RL     76
CP      5
C       2
A       1
RC      1
E       1
CC      1
Name: LU, dtype: int64

It appears to be empty parcels of land, for the most part. In this case, there *are* no buildings, so of course building type is null. This means we can just fill in the empty data with something like "NB - No Building".

In [18]:
cleaned["BLDG_TYPE"] = cleaned["BLDG_TYPE"].fillna("NB - No Building")

Okay, the next set to take care of is properties with no LAND_VALUE, BLDG_VALUE, or TOTAL_VALUE. I will simply drop these because I know they are just untaxable land/government properties. 

In [19]:
cleaned.drop(cleaned.index[cleaned["LAND_VALUE"].isnull()], inplace=True)

In [20]:
cleaned.isnull().sum()

PID                     0
ST_NUM                  0
ST_NAME                 0
ZIPCODE                 0
NUM_BLDGS               0
LU                      0
BLDG_TYPE               0
OWN_OCC                 0
RES_FLOOR           11016
CD_FLOOR            86339
RES_UNITS               0
COM_UNITS               0
RC_UNITS                0
LAND_SF                 0
GROSS_AREA              0
LIVING_AREA             0
LAND_VALUE              0
BLDG_VALUE              0
TOTAL_VALUE             0
GROSS_TAX               0
YR_BUILT            10905
YR_REMODEL          71701
STRUCTURE_CLASS     77802
ROOF_STRUCTURE      24454
ROOF_COVER          24455
INT_WALL            24811
EXT_FINISHED        11047
INT_COND            24815
EXT_COND            24424
OVERALL_COND         7089
BED_RMS                 0
FULL_BTH                0
HLF_BTH                 0
KITCHEN                 0
TT_RMS                  0
BDRM_COND           86634
BTHRM_STYLE1        24725
BTHRM_STYLE2        71840
BTHRM_STYLE3

The next issue to handle is floor data. Most likely, these will have to be filled in with 1 or something like that.

In [21]:
cleaned[cleaned["RES_FLOOR"].isnull()]["LU"].value_counts()

CP    5758
E     2585
CL    1045
C      809
CC     269
RC     167
I      116
A      113
RL      79
R4      42
EA      30
CM       1
R2       1
AH       1
Name: LU, dtype: int64

The most common type of property with no residential floors is parking. That makes sense, but the remaining types don't really make sense, as some of them are still places that would have residential floors.

In [22]:
cleaned[cleaned["CD_FLOOR"].isnull()]["LU"].value_counts()

R1    30455
R2    16954
R3    13585
CP     5795
E      4938
C      4356
RC     2799
A      2586
R4     2529
CL     1063
I       406
CC      316
CM      264
EA      212
RL       79
CD        1
AH        1
Name: LU, dtype: int64

For condominium floor, it makes sense that residential homes don't count and thus have no data, but this does mean we have no floor data for most of the proprties. This means that the best choice is just to drop floor data entirely.

In [23]:
cleaned.drop(["CD_FLOOR","RES_FLOOR"],axis=1, inplace=True)

In [24]:
cleaned.isnull().sum()

PID                     0
ST_NUM                  0
ST_NAME                 0
ZIPCODE                 0
NUM_BLDGS               0
LU                      0
BLDG_TYPE               0
OWN_OCC                 0
RES_UNITS               0
COM_UNITS               0
RC_UNITS                0
LAND_SF                 0
GROSS_AREA              0
LIVING_AREA             0
LAND_VALUE              0
BLDG_VALUE              0
TOTAL_VALUE             0
GROSS_TAX               0
YR_BUILT            10905
YR_REMODEL          71701
STRUCTURE_CLASS     77802
ROOF_STRUCTURE      24454
ROOF_COVER          24455
INT_WALL            24811
EXT_FINISHED        11047
INT_COND            24815
EXT_COND            24424
OVERALL_COND         7089
BED_RMS                 0
FULL_BTH                0
HLF_BTH                 0
KITCHEN                 0
TT_RMS                  0
BDRM_COND           86634
BTHRM_STYLE1        24725
BTHRM_STYLE2        71840
BTHRM_STYLE3       119608
KITCHEN_TYPE        24842
KITCHEN_STYL

Now, we need to contend with missing data for the year of construction, as well as renovation year. Blank renovation years could just be replaced with the original construction year.

In [25]:
cleaned[cleaned["YR_BUILT"].isnull()]["LU"].value_counts()

CP    5758
E     2528
CL    1046
C      793
CC     267
RC     130
I      104
RL      79
A       77
R4      39
CD      37
EA      17
R3      10
CM       8
R1       7
R2       4
AH       1
Name: LU, dtype: int64

It might just be better to drop cells without a construction year.

In [26]:
cleaned.drop(cleaned.index[cleaned["YR_BUILT"].isnull()], inplace=True)

Now, we have to fill in renovation years with the original year of construction.

In [27]:
cleaned["YR_REMODEL"].fillna(cleaned["YR_BUILT"],inplace=True)

In [28]:
cleaned.isnull().sum()

PID                     0
ST_NUM                  0
ST_NAME                 0
ZIPCODE                 0
NUM_BLDGS               0
LU                      0
BLDG_TYPE               0
OWN_OCC                 0
RES_UNITS               0
COM_UNITS               0
RC_UNITS                0
LAND_SF                 0
GROSS_AREA              0
LIVING_AREA             0
LAND_VALUE              0
BLDG_VALUE              0
TOTAL_VALUE             0
GROSS_TAX               0
YR_BUILT                0
YR_REMODEL              0
STRUCTURE_CLASS     66949
ROOF_STRUCTURE      13617
ROOF_COVER          13619
INT_WALL            13966
EXT_FINISHED          220
INT_COND            13970
EXT_COND            13588
OVERALL_COND         7072
BED_RMS                 0
FULL_BTH                0
HLF_BTH                 0
KITCHEN                 0
TT_RMS                  0
BDRM_COND           75764
BTHRM_STYLE1        13880
BTHRM_STYLE2        60990
BTHRM_STYLE3       108736
KITCHEN_TYPE        13998
KITCHEN_STYL

Next, we have to fill in blank values for structure class. Let's see what these values can be.

In [29]:
cleaned['STRUCTURE_CLASS'].unique()

array(['R - ', nan, 'D - Wood/Frame', 'C - Brick/Concr',
       'B - Reinf Concr', 'A - Struct Steel', 'E - Metal'], dtype=object)

In [30]:
cleaned[cleaned["STRUCTURE_CLASS"].isnull()]["LU"].value_counts()

CD    65428
R1      480
E       247
CM      237
R2      203
R3      181
A        59
RC       37
C        23
CC       18
R4       12
EA       12
I        12
Name: LU, dtype: int64

So, most of the missing values for structure class are condominium-style buildings. The rest are a mixture of residential and unclear properties. We can make a new label specifically for condos, label the residental ones with "R", and then remove the remaining ones. 

In [31]:
mask = cleaned.index[(cleaned["STRUCTURE_CLASS"].isnull()) & (cleaned["LU"] == "CD")]
cleaned.loc[mask,"STRUCTURE_CLASS"] = cleaned.loc[mask,"STRUCTURE_CLASS"].fillna("CD - Condominium")

In [32]:
classes = ["R1","R2","R3","R4","RC","A","CM"]
mask = cleaned.index[(cleaned["STRUCTURE_CLASS"].isnull()) & (cleaned["LU"].isin(classes))]
cleaned.loc[mask,"STRUCTURE_CLASS"] = cleaned.loc[mask,"STRUCTURE_CLASS"].fillna("R - ")

We can now drop the remaining rows with no STRUCTURE_CLASS.

In [33]:
cleaned.drop(cleaned.index[cleaned["STRUCTURE_CLASS"].isnull()], inplace=True)

Let's see what's next.

In [34]:
cleaned.isnull().sum()

PID                     0
ST_NUM                  0
ST_NAME                 0
ZIPCODE                 0
NUM_BLDGS               0
LU                      0
BLDG_TYPE               0
OWN_OCC                 0
RES_UNITS               0
COM_UNITS               0
RC_UNITS                0
LAND_SF                 0
GROSS_AREA              0
LIVING_AREA             0
LAND_VALUE              0
BLDG_VALUE              0
TOTAL_VALUE             0
GROSS_TAX               0
YR_BUILT                0
YR_REMODEL              0
STRUCTURE_CLASS         0
ROOF_STRUCTURE      13514
ROOF_COVER          13516
INT_WALL            13855
EXT_FINISHED          123
INT_COND            13858
EXT_COND            13486
OVERALL_COND         7066
BED_RMS                 0
FULL_BTH                0
HLF_BTH                 0
KITCHEN                 0
TT_RMS                  0
BDRM_COND           75635
BTHRM_STYLE1        13767
BTHRM_STYLE2        60718
BTHRM_STYLE3       108432
KITCHEN_TYPE        13876
KITCHEN_STYL

ROOF_STRUCTURE and ROOF_COVER are next. The missing values will likely be the same parcels.

In [35]:
cleaned["ROOF_STRUCTURE"].unique()

array(['F - Flat', 'M - Mansard', 'H - Hip', 'G - Gable', nan,
       'L - Gambrel', 'S - Shed', 'O - Other'], dtype=object)

In [36]:
cleaned[cleaned["ROOF_STRUCTURE"].isnull()]["LU"].value_counts()

C     3533
RC    2551
R4    2465
A     2319
E     2166
I      290
EA     132
CD      28
CL      18
CC      12
Name: LU, dtype: int64

There doesn't seem to be a clear explanation as to why this data is missing for some properties. I will just fill it in with "U - Unknown". I think this information is important enough to keep, but given the number of properties with this data missing, I also don't want to just remove the ones where it is missing.

In [37]:
cleaned["ROOF_COVER"].unique()

array(['C - Composition', 'R - Rubber Roof', 'S - Slate',
       'A - Asphalt Shingl', 'O - Other', nan, 'W - Wood Shingle',
       'T - Tile'], dtype=object)

In [38]:
cleaned[cleaned["ROOF_COVER"].isnull()]["LU"].value_counts()

C     3533
RC    2551
R4    2465
A     2319
E     2165
I      290
EA     132
CD      31
CL      18
CC      12
Name: LU, dtype: int64

In [39]:
cleaned["ROOF_COVER"].fillna("U - Unknown", inplace=True)
cleaned["ROOF_STRUCTURE"].fillna("U - Unknown", inplace=True)

In [40]:
cleaned.isnull().sum()

PID                     0
ST_NUM                  0
ST_NAME                 0
ZIPCODE                 0
NUM_BLDGS               0
LU                      0
BLDG_TYPE               0
OWN_OCC                 0
RES_UNITS               0
COM_UNITS               0
RC_UNITS                0
LAND_SF                 0
GROSS_AREA              0
LIVING_AREA             0
LAND_VALUE              0
BLDG_VALUE              0
TOTAL_VALUE             0
GROSS_TAX               0
YR_BUILT                0
YR_REMODEL              0
STRUCTURE_CLASS         0
ROOF_STRUCTURE          0
ROOF_COVER              0
INT_WALL            13855
EXT_FINISHED          123
INT_COND            13858
EXT_COND            13486
OVERALL_COND         7066
BED_RMS                 0
FULL_BTH                0
HLF_BTH                 0
KITCHEN                 0
TT_RMS                  0
BDRM_COND           75635
BTHRM_STYLE1        13767
BTHRM_STYLE2        60718
BTHRM_STYLE3       108432
KITCHEN_TYPE        13876
KITCHEN_STYL

Let's take care of INT_WALL. (and likely drop rows missing EXT_FINISHED)

In [41]:
cleaned["INT_WALL"].value_counts()

N - Normal         125758
E - Elaborate        4314
S - Substandard        78
G - Good                2
Name: INT_WALL, dtype: int64

It might just make sense to fill in missing values with "U - Unknown".  

In [42]:
cleaned["EXT_FINISHED"].value_counts()

B - Brick/Stone       47081
M - Vinyl             39915
W - Wood Shake        15441
F - Frame/Clapbrd     11759
01 - Brick             7759
C - Cement Board       5638
A - Asbestos           3868
G - Glass              2826
09 - Wood Siding       1798
S - Stucco             1227
03 - Poured Concr      1066
P - Asphalt             943
10 - Alum/Vinyl         913
12 - Conc Block         752
V - Brck/Stn Venr       685
U - Alum Siding         682
02 - Stone              342
O - Other               249
11 - Metal Siding       195
K - Concrete            164
04 - Precast Concr      154
08 - Stucco             143
06 - Metal/Glass        130
05 - Concr & Glass       63
15 - Corrug Siding       55
07 - Stone/Marble        32
14 - Hollow Tile          3
13 - Br Sill/Sash         1
Name: EXT_FINISHED, dtype: int64

EXT_FINISHED could just be filled with O - Other

In [43]:
cleaned["INT_WALL"].fillna("U - Unknown",inplace=True)
cleaned["EXT_FINISHED"].fillna("O - Other",inplace=True)

In [44]:
cleaned.isnull().sum()

PID                     0
ST_NUM                  0
ST_NAME                 0
ZIPCODE                 0
NUM_BLDGS               0
LU                      0
BLDG_TYPE               0
OWN_OCC                 0
RES_UNITS               0
COM_UNITS               0
RC_UNITS                0
LAND_SF                 0
GROSS_AREA              0
LIVING_AREA             0
LAND_VALUE              0
BLDG_VALUE              0
TOTAL_VALUE             0
GROSS_TAX               0
YR_BUILT                0
YR_REMODEL              0
STRUCTURE_CLASS         0
ROOF_STRUCTURE          0
ROOF_COVER              0
INT_WALL                0
EXT_FINISHED            0
INT_COND            13858
EXT_COND            13486
OVERALL_COND         7066
BED_RMS                 0
FULL_BTH                0
HLF_BTH                 0
KITCHEN                 0
TT_RMS                  0
BDRM_COND           75635
BTHRM_STYLE1        13767
BTHRM_STYLE2        60718
BTHRM_STYLE3       108432
KITCHEN_TYPE        13876
KITCHEN_STYL

The next variables to fill in are INT_COND, EXT_COND and OVERALL_COND.

In [45]:
cleaned["EXT_COND"].value_counts()

A - Average      74921
G - Good         47658
E - Excellent     5682
F - Fair          2192
P - Poor            68
Name: EXT_COND, dtype: int64

In [46]:
cleaned["INT_COND"].value_counts()

A - Average      64420
G - Good         53068
E - Excellent    11362
F - Fair          1210
P - Poor            89
Name: INT_COND, dtype: int64

In [47]:
cleaned["OVERALL_COND"].value_counts()

A - Average                103166
G - Good                    29076
VG - Very Good               1343
E - Excellent                1149
EX - Excellent               1060
F - Fair                     1017
P - Poor                       98
US - Unsound                   20
VP - Very Poor                 11
AVG - Default - Average         1
Name: OVERALL_COND, dtype: int64

Because OVERALL_COND has less missing values overall, it probably is best to just ignore/drop EXT_COND and INT_COND and assume that OVERALL_COND captures that information well enough. In general, interior condition and exterior condition seem to be relatively similar anyway.

In [48]:
cleaned.drop(["EXT_COND", "INT_COND"],axis = 1,inplace=True)

In [49]:
cleaned["OVERALL_COND"].fillna("AVG - Default - Average",inplace=True)

In [50]:
cleaned.isnull().sum()

PID                     0
ST_NUM                  0
ST_NAME                 0
ZIPCODE                 0
NUM_BLDGS               0
LU                      0
BLDG_TYPE               0
OWN_OCC                 0
RES_UNITS               0
COM_UNITS               0
RC_UNITS                0
LAND_SF                 0
GROSS_AREA              0
LIVING_AREA             0
LAND_VALUE              0
BLDG_VALUE              0
TOTAL_VALUE             0
GROSS_TAX               0
YR_BUILT                0
YR_REMODEL              0
STRUCTURE_CLASS         0
ROOF_STRUCTURE          0
ROOF_COVER              0
INT_WALL                0
EXT_FINISHED            0
OVERALL_COND            0
BED_RMS                 0
FULL_BTH                0
HLF_BTH                 0
KITCHEN                 0
TT_RMS                  0
BDRM_COND           75635
BTHRM_STYLE1        13767
BTHRM_STYLE2        60718
BTHRM_STYLE3       108432
KITCHEN_TYPE        13876
KITCHEN_STYLE1      13873
KITCHEN_STYLE2     112169
KITCHEN_STYL

The next variable with missing data is BDRM_COND.

In [51]:
cleaned["BDRM_COND"].value_counts()

Average      53123
Good         13355
Excellent      966
Fair           865
Poor            63
Name: BDRM_COND, dtype: int64

In [52]:
cleaned[cleaned["BDRM_COND"].isnull()]["LU"].value_counts()

R1    30447
R2    16946
R3    13575
C      3539
RC     2670
A      2510
R4     2490
E      2344
I       290
CD      289
CM      256
EA      185
CP       40
CC       36
CL       18
Name: LU, dtype: int64

Unexpectedly, the vast majority of properties without bedroom condition information are residential properties. And because there are so many rows without this information, it makes sense to drop the variable entirely.

In [53]:
cleaned.drop("BDRM_COND",inplace=True,axis=1)

Let's look at what's going on with the bathrooms next.

In [54]:
cleaned["BTHRM_STYLE3"].value_counts()

M - Modern           17489
S - Semi-Modern      14402
N - No Remodeling     2020
L - Luxury            1664
Name: BTHRM_STYLE3, dtype: int64

In [55]:
cleaned[cleaned["BTHRM_STYLE1"].isnull()]["LU"].value_counts()

C     3537
RC    2589
R4    2480
A     2333
E     2166
I      290
CM     147
EA     136
CP      37
CC      34
CL      18
Name: LU, dtype: int64

I think it's fair to assume that a missing bathroom style (1, 2 or 3) means that a property does not have that many bathrooms. I will create a new entry for this variable, "NB - No Bathroom".

In [56]:
cleaned["BTHRM_STYLE1"].fillna("NB - No Bathroom",inplace=True)
cleaned["BTHRM_STYLE2"].fillna("NB - No Bathroom",inplace=True)
cleaned["BTHRM_STYLE3"].fillna("NB - No Bathroom",inplace=True)

This same logic probably applies to kitchens as well.

In [57]:
cleaned["KITCHEN_TYPE"].value_counts()

O - One Person                 42112
1F - 1 Full Eat In Kitchens    29416
F - Full Eat In                23688
2F - 2 Full Eat In Kitchens    17054
3F - 3 Full Eat In Kitchens    12310
P - Pullman                     2741
0F - 0 Full Eat In Kitchens     2664
N - None                         117
4F - 4 Full Eat In Kitchens       28
5F - 5 Full Eat In Kitchens        1
Name: KITCHEN_TYPE, dtype: int64

What types of bulidings have NAN for kitchen type?

In [58]:
cleaned[cleaned["KITCHEN_TYPE"].isna()]["LU"].value_counts()

C     3537
RC    2589
R4    2480
A     2333
E     2166
I      290
CM     248
EA     136
CP      37
CC      35
CL      18
CD       2
R2       2
R3       2
R1       1
Name: LU, dtype: int64

It seems that mainly commerical properties have no kitchens, which makes sense. We can just fill in missing data with "N - None". (TODO: notice that R4 also has missing data, so this would be best filled in differently.)

In [59]:
cleaned["KITCHEN_TYPE"].fillna("N - None",inplace=True)

In [60]:
cleaned["KITCHEN_STYLE2"].value_counts()

S - Semi-Modern      18807
M - Modern            9952
N - No Remodeling     2997
L - Luxury              82
Name: KITCHEN_STYLE2, dtype: int64

In [61]:
cleaned[cleaned["KITCHEN_STYLE1"].isnull()]["LU"].value_counts()

C     3537
RC    2589
R4    2479
A     2335
E     2167
I      290
CM     248
EA     136
CP      37
CC      35
CL      18
CD       2
Name: LU, dtype: int64

Like with bathrooms, missing data for a specific number of kitchen style likely just means that the property doesn't have that many kitchens.

In [62]:
cleaned["KITCHEN_STYLE1"].fillna("NK - No Kitchen",inplace=True)
cleaned["KITCHEN_STYLE2"].fillna("NK - No Kitchen",inplace=True)
cleaned["KITCHEN_STYLE3"].fillna("NK - No Kitchen",inplace=True)

In [63]:
cleaned.isna().sum()

PID                    0
ST_NUM                 0
ST_NAME                0
ZIPCODE                0
NUM_BLDGS              0
LU                     0
BLDG_TYPE              0
OWN_OCC                0
RES_UNITS              0
COM_UNITS              0
RC_UNITS               0
LAND_SF                0
GROSS_AREA             0
LIVING_AREA            0
LAND_VALUE             0
BLDG_VALUE             0
TOTAL_VALUE            0
GROSS_TAX              0
YR_BUILT               0
YR_REMODEL             0
STRUCTURE_CLASS        0
ROOF_STRUCTURE         0
ROOF_COVER             0
INT_WALL               0
EXT_FINISHED           0
OVERALL_COND           0
BED_RMS                0
FULL_BTH               0
HLF_BTH                0
KITCHEN                0
TT_RMS                 0
BTHRM_STYLE1           0
BTHRM_STYLE2           0
BTHRM_STYLE3           0
KITCHEN_TYPE           0
KITCHEN_STYLE1         0
KITCHEN_STYLE2         0
KITCHEN_STYLE3         0
HEAT_TYPE          13859
HEAT_FUEL          75354


Now let's look at HEAT_TYPE, HEAT_FUEL, and AC_TYPE. Properties with no AC_TYPE are likely the same as properties without HEAT_TYPE.

In [64]:
cleaned["HEAT_TYPE"].value_counts()

W - Ht Water/Steam    71739
F - Forced Hot Air    47131
E - Electric           5868
P - Heat Pump          4596
S - Space Heat          685
N - None                 89
O - Other                40
Name: HEAT_TYPE, dtype: int64

In [65]:
cleaned["HEAT_FUEL"].value_counts()

I - Indiv. Cntrl      42215
Y - Self Contained    17437
C - Common             8031
N - None                970
Name: HEAT_FUEL, dtype: int64

In [66]:
cleaned["AC_TYPE"].value_counts()

N - None           79796
C - Central AC     49459
D - Ductless AC      891
Y - Yes                1
Name: AC_TYPE, dtype: int64

HEAT_FUEL seems mostly pointless and difficult to fill in, so it will simply be dropped.

In [67]:
cleaned[cleaned["AC_TYPE"].isna()]["LU"].value_counts()

C     3539
RC    2588
R4    2479
A     2330
E     2166
I      290
CM     245
EA     136
CP      35
CC      34
CL      18
Name: LU, dtype: int64

In [68]:
cleaned[cleaned["HEAT_TYPE"].isna()]["LU"].value_counts()

C     3538
RC    2588
R4    2479
A     2331
E     2166
I      290
CM     244
EA     136
CP      35
CC      34
CL      18
Name: LU, dtype: int64

These are most definitely the same properties and they seem to mainly be commercial/mixed properties. It is very strange for buildings in Boston to not have heat, though. For AC_TYPE, I will fill in the data as "N - None". For HEAT_TYPE, however, I will fill in the data with "U - Unknown" because I think this many buildings definitely couldn't be lacking heat.

In [69]:
cleaned.drop("HEAT_FUEL",axis = 1, inplace=True)
cleaned["AC_TYPE"].fillna("N - None", inplace=True)
cleaned["HEAT_TYPE"].fillna("U - Unknown", inplace=True)

In [70]:
cleaned.isna().sum()

PID                    0
ST_NUM                 0
ST_NAME                0
ZIPCODE                0
NUM_BLDGS              0
LU                     0
BLDG_TYPE              0
OWN_OCC                0
RES_UNITS              0
COM_UNITS              0
RC_UNITS               0
LAND_SF                0
GROSS_AREA             0
LIVING_AREA            0
LAND_VALUE             0
BLDG_VALUE             0
TOTAL_VALUE            0
GROSS_TAX              0
YR_BUILT               0
YR_REMODEL             0
STRUCTURE_CLASS        0
ROOF_STRUCTURE         0
ROOF_COVER             0
INT_WALL               0
EXT_FINISHED           0
OVERALL_COND           0
BED_RMS                0
FULL_BTH               0
HLF_BTH                0
KITCHEN                0
TT_RMS                 0
BTHRM_STYLE1           0
BTHRM_STYLE2           0
BTHRM_STYLE3           0
KITCHEN_TYPE           0
KITCHEN_STYLE1         0
KITCHEN_STYLE2         0
KITCHEN_STYLE3         0
HEAT_TYPE              0
AC_TYPE                0


In [71]:
cleaned["PROP_VIEW"].value_counts()

A - Average      107403
G - Good          11874
F - Fair           6037
E - Excellent      4149
P - Poor            530
S - Special         284
Name: PROP_VIEW, dtype: int64

Admittedly, I don't know very much about the meaning of this variable. Additionally, because more than half of the data is the same value, I think I can just drop it from the analysis.

In [72]:
cleaned.drop("PROP_VIEW",axis=1,inplace=True)

The last variable to fill in is CORNER_UNIT

In [73]:
cleaned["CORNER_UNIT"].value_counts()

N - No     55727
Y - Yes    12935
Name: CORNER_UNIT, dtype: int64

In [74]:
cleaned["CORNER_UNIT"].fillna("N - No",inplace=True)

In [75]:
cleaned.isna().sum()

PID                0
ST_NUM             0
ST_NAME            0
ZIPCODE            0
NUM_BLDGS          0
LU                 0
BLDG_TYPE          0
OWN_OCC            0
RES_UNITS          0
COM_UNITS          0
RC_UNITS           0
LAND_SF            0
GROSS_AREA         0
LIVING_AREA        0
LAND_VALUE         0
BLDG_VALUE         0
TOTAL_VALUE        0
GROSS_TAX          0
YR_BUILT           0
YR_REMODEL         0
STRUCTURE_CLASS    0
ROOF_STRUCTURE     0
ROOF_COVER         0
INT_WALL           0
EXT_FINISHED       0
OVERALL_COND       0
BED_RMS            0
FULL_BTH           0
HLF_BTH            0
KITCHEN            0
TT_RMS             0
BTHRM_STYLE1       0
BTHRM_STYLE2       0
BTHRM_STYLE3       0
KITCHEN_TYPE       0
KITCHEN_STYLE1     0
KITCHEN_STYLE2     0
KITCHEN_STYLE3     0
HEAT_TYPE          0
AC_TYPE            0
FIRE_PLACE         0
NUM_PARKING        0
CORNER_UNIT        0
dtype: int64

All of the data is now filled in!

In [76]:
cleaned

,PID,ST_NUM,ST_NAME,ZIPCODE,NUM_BLDGS,LU,BLDG_TYPE,OWN_OCC,RES_UNITS,COM_UNITS,...,BTHRM_STYLE3,KITCHEN_TYPE,KITCHEN_STYLE1,KITCHEN_STYLE2,KITCHEN_STYLE3,HEAT_TYPE,AC_TYPE,FIRE_PLACE,NUM_PARKING,CORNER_UNIT
0,100001000,104 A 104,PUTNAM ST,2128,1,R3,RE - Row End,Y,0,0,...,S - Semi-Modern,3F - 3 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,N - None,0,3,N - No
1,100002000,197,LEXINGTON ST,2128,1,R3,RM - Row Middle,Y,0,0,...,M - Modern,3F - 3 Full Eat In Kitchens,M - Modern,M - Modern,M - Modern,F - Forced Hot Air,C - Central AC,0,0,N - No
2,100003000,199,LEXINGTON ST,2128,1,R3,RM - Row Middle,N,0,0,...,M - Modern,3F - 3 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,S - Space Heat,N - None,0,0,N - No
3,100004000,201,LEXINGTON ST,2128,1,R3,RM - Row Middle,N,0,0,...,S - Semi-Modern,3F - 3 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,N - None,0,0,N - No
4,100005000,203,LEXINGTON ST,2128,1,R2,RE - Row End,Y,0,0,...,S - Semi-Modern,2F - 2 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,NK - No Kitchen,W - Ht Water/Steam,N - None,0,0,N - No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178591,2205665002,14,LAKE ST,2135,1,CD,FS - Free Standing,N,1,0,...,NB - No Bathroom,F - Full Eat In,S - Semi-Modern,NK - No Kitchen,NK - No Kitchen,W - Ht Water/Steam,N - None,1,1,N - No
178592,2205665004,12,LAKE ST,2135,1,CD,FS - Free Standing,N,1,0,...,NB - No Bathroom,F - Full Eat In,S - Semi-Modern,NK - No Kitchen,NK - No Kitchen,W - Ht Water/Steam,N - None,1,1,N - No
178595,2205668000,4,LAKE ST,2135,1,R3,CV - Conventional,N,0,0,...,M - Modern,2F - 2 Full Eat In Kitchens,M - Modern,M - Modern,M - Modern,W - Ht Water/Steam,N - None,0,2,N - No
178596,2205669000,2193 2201,COMMONWEALTH AV,2135,1,C,319 - STRIP RETAIL/ OFFICE,N,0,8,...,NB - No Bathroom,N - None,NK - No Kitchen,NK - No Kitchen,NK - No Kitchen,U - Unknown,N - None,0,0,N - No


There is one last thing to do. The prices are not formatted properly as machine-readble numbers, they are human-readable with commas in them. Those commas must be removed before doing any analysis.

In [77]:
cleaned["BLDG_VALUE"].describe()

count     144007
unique     24997
top            0
freq         260
Name: BLDG_VALUE, dtype: object

These are the variables that must be reformatted:
LAND_SF
GROSS_AREA
LIVING_AREA
LAND_VALUE
BLDG_VALUE
TOTAL_VALUE
GROSS_TAX

In [78]:
columns_to_fix = ["LAND_VALUE","BLDG_VALUE","TOTAL_VALUE","GROSS_TAX"]
#cleaned[columns_to_fix].astype(str).replace(",","")
#pd.to_numeric(cleaned[columns_to_fix],errors='coerce')
#cleaned[columns_to_fix].filter(like="num").apply(lambda x: x.str.replace('^[0-9.]',''))
#cleaned.apply(lambda x: x.astype(str).replace(',','').astype(float))
for col in columns_to_fix:
  cleaned[col] = cleaned[col].str.replace(',','')
  cleaned[col] = cleaned[col].str.replace('$','')
  cleaned[col] = pd.to_numeric(cleaned[col],errors='coerce')
cleaned[columns_to_fix]
#data["TOTAL_VALUE"] = data["TOTAL_VALUE"].str.replace(',','')
#data["TOTAL_VALUE"] = pd.to_numeric(data["TOTAL_VALUE"],errors='coerce')

/tmp/ipykernel_1431/1914829922.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cleaned[col] = cleaned[col].str.replace('$','')


,LAND_VALUE,BLDG_VALUE,TOTAL_VALUE,GROSS_TAX
0,179500,539900,719400,7827.07
1,180300,564500,744800,8103.42
2,180800,549700,730500,7947.84
3,181400,486500,667900,7266.75
4,210900,531800,742700,8080.57
...,...,...,...,...
178591,0,519200,519200,5648.90
178592,0,471200,471200,5126.66
178595,442300,810000,1252300,13625.02
178596,990900,1426600,2427000,60626.46


In [86]:
cleaned["TOTAL_VALUE"].describe()

count    1.440070e+05
mean     1.585295e+06
std      1.408356e+07
min      0.000000e+00
25%      4.716000e+05
50%      6.474000e+05
75%      9.358000e+05
max      2.020459e+09
Name: TOTAL_VALUE, dtype: float64

In [82]:
cleaned["LAND_VALUE"].describe()

count    1.440070e+05
mean     3.905115e+05
std      4.467598e+06
min      0.000000e+00
25%      0.000000e+00
50%      1.183000e+05
75%      2.236000e+05
max      4.860469e+08
Name: LAND_VALUE, dtype: float64

In [83]:
cleaned["BLDG_VALUE"].describe()

count    1.440070e+05
mean     1.192055e+06
std      1.036641e+07
min      0.000000e+00
25%      3.761000e+05
50%      5.250000e+05
75%      7.729000e+05
max      1.600662e+09
Name: BLDG_VALUE, dtype: float64

In [87]:
cleaned

,PID,ST_NUM,ST_NAME,ZIPCODE,NUM_BLDGS,LU,BLDG_TYPE,OWN_OCC,RES_UNITS,COM_UNITS,...,BTHRM_STYLE3,KITCHEN_TYPE,KITCHEN_STYLE1,KITCHEN_STYLE2,KITCHEN_STYLE3,HEAT_TYPE,AC_TYPE,FIRE_PLACE,NUM_PARKING,CORNER_UNIT
0,100001000,104 A 104,PUTNAM ST,2128,1,R3,RE - Row End,Y,0,0,...,S - Semi-Modern,3F - 3 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,N - None,0,3,N - No
1,100002000,197,LEXINGTON ST,2128,1,R3,RM - Row Middle,Y,0,0,...,M - Modern,3F - 3 Full Eat In Kitchens,M - Modern,M - Modern,M - Modern,F - Forced Hot Air,C - Central AC,0,0,N - No
2,100003000,199,LEXINGTON ST,2128,1,R3,RM - Row Middle,N,0,0,...,M - Modern,3F - 3 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,S - Space Heat,N - None,0,0,N - No
3,100004000,201,LEXINGTON ST,2128,1,R3,RM - Row Middle,N,0,0,...,S - Semi-Modern,3F - 3 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,N - None,0,0,N - No
4,100005000,203,LEXINGTON ST,2128,1,R2,RE - Row End,Y,0,0,...,S - Semi-Modern,2F - 2 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,NK - No Kitchen,W - Ht Water/Steam,N - None,0,0,N - No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178591,2205665002,14,LAKE ST,2135,1,CD,FS - Free Standing,N,1,0,...,NB - No Bathroom,F - Full Eat In,S - Semi-Modern,NK - No Kitchen,NK - No Kitchen,W - Ht Water/Steam,N - None,1,1,N - No
178592,2205665004,12,LAKE ST,2135,1,CD,FS - Free Standing,N,1,0,...,NB - No Bathroom,F - Full Eat In,S - Semi-Modern,NK - No Kitchen,NK - No Kitchen,W - Ht Water/Steam,N - None,1,1,N - No
178595,2205668000,4,LAKE ST,2135,1,R3,CV - Conventional,N,0,0,...,M - Modern,2F - 2 Full Eat In Kitchens,M - Modern,M - Modern,M - Modern,W - Ht Water/Steam,N - None,0,2,N - No
178596,2205669000,2193 2201,COMMONWEALTH AV,2135,1,C,319 - STRIP RETAIL/ OFFICE,N,0,8,...,NB - No Bathroom,N - None,NK - No Kitchen,NK - No Kitchen,NK - No Kitchen,U - Unknown,N - None,0,0,N - No


This data is now ready for analysis!